In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/spam-filter/emails.csv


In [19]:
ls

__notebook_source__.ipynb  model.h5  model_in_json.json


In [3]:
#pandas for dataframing 
#use keras for modeling
import pandas as pd 
import keras

Using TensorFlow backend.


In [4]:
#the spam email filter
df = pd.read_csv("../input/spam-filter/emails.csv")

In [ ]:
df.head()

In [6]:
text_messages = df['text']

In [7]:
#preprocessed the emails using reggex
preprocessed = text_messages.str.replace(r'^.+@[^\.].*\.[a-z]{2,}$', 'emailaddr')
preprocessed = preprocessed.str.replace(r'^(http(s?)\:\/\/)*[0-9a-zA-Z]([-.\w]*[0-9a-zA-Z])*(:(0-9)*)*(\/?)([a-zA-Z0-9\-\.\?\,\'\/\\\+&amp;%\$#_]*)?', 'webaddr')
preprocessed = preprocessed.str.replace(r"\(([0-9]{2}|0{1}((x|[0-9]){2}[0-9]{2}))\)\s*[0-9]{3,4}[- ]*[0-9]{4}", 'phonenum')

In [8]:
preprocessed = preprocessed.str.replace(r'[^\w\d\s]', ' ')
# replace double spaces
preprocessed = preprocessed.str.replace(r'\s+', ' ')
preprocessed = preprocessed.str.replace(r'_', ' ')

#no begin or end whitespaces
preprocessed = preprocessed.str.replace(r'^\s+|\s+?$', '')

In [9]:
# Remove word stems using a Porter stemmer 
#keep only the root word
import nltk
ps = nltk.PorterStemmer()

preprocessed =preprocessed.apply(lambda x: ' '.join(ps.stem(term) for term in x.split()))

In [10]:
#we have the stop words deleter imbeded in Count Vectorizer
#It return a matrix of counter tokens having the max_features attr

from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
 
vectorizer = CountVectorizer(binary=True, stop_words=stopwords.words('english'), 
                             lowercase=False, min_df=3, max_df=0.9, max_features=1500)
preprocessed1 = vectorizer.fit_transform(preprocessed)

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_text, y_train,y_test = train_test_split(preprocessed1, df['spam'], random_state= 42, test_size = 0.2)

In [12]:
from keras.layers import Dropout
help(Dropout)

Help on class Dropout in module keras.layers.core:

class Dropout(keras.engine.base_layer.Layer)
 |  Applies Dropout to the input.
 |  
 |  Dropout consists in randomly setting
 |  a fraction `rate` of input units to 0 at each update during training time,
 |  which helps prevent overfitting.
 |  
 |  # Arguments
 |      rate: float between 0 and 1. Fraction of the input units to drop.
 |      noise_shape: 1D integer tensor representing the shape of the
 |          binary dropout mask that will be multiplied with the input.
 |          For instance, if your inputs have shape
 |          `(batch_size, timesteps, features)` and
 |          you want the dropout mask to be the same for all timesteps,
 |          you can use `noise_shape=(batch_size, 1, features)`.
 |      seed: A Python integer to use as random seed.
 |  
 |  # References
 |      - [Dropout: A Simple Way to Prevent Neural Networks from Overfitting](
 |         http://www.jmlr.org/papers/volume15/srivastava14a/srivastava14a.

In [13]:
#Build keras model
from keras.models import Sequential
from keras.layers import Dense
 
model = Sequential()
 
model.add(Dense(units=500, activation='relu', input_dim=len(vectorizer.get_feature_names())))
model.add(Dropout(rate = 0.3))
model.add(Dense(units=1, activation='sigmoid'))
 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 500)               750500    
_________________________________________________________________
dropout_1 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 501       
Total params: 751,001
Trainable params: 751,001
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(X_train[:-100], y_train[:-100], 
          epochs=2, batch_size=128, verbose=1, 
          validation_data=(X_train[-100:], y_train[-100:]))
 

Train on 4482 samples, validate on 100 samples
Epoch 1/2
4482/4482 [==============================] - 1s 201us/step - loss: 0.2161 - accuracy: 0.9132 - val_loss: 0.0658 - val_accuracy: 1.0000
Epoch 2/2
4482/4482 [==============================] - 1s 149us/step - loss: 0.0464 - accuracy: 0.9909 - val_loss: 0.0299 - val_accuracy: 1.0000


In [15]:

scores = model.evaluate(X_text, y_test, verbose=1)
print("Accuracy:", scores[1])  # Accuracy: 0.875

 

1146/1146 [==============================] - 0s 95us/step
Accuracy: 0.9860383868217468


In [26]:
import pickle
#let's fit the vectorizer on the dataset words 
vectrizer = vectorizer.fit(preprocessed)

#now pickle
pickle.dump(vectorizer, open("vector.pickel", "wb"))

In [27]:
import json
# lets save the main model
model_json = model.to_json()
with open("model_in_json.json", "w") as json_file:
    json.dump(model_json, json_file)

model.save_weights("model.h5")

In [38]:
#let's create a link to download the main model 
#and vectorizer model
from IPython.display import FileLink
#create link for the model's weights
FileLink("model.h5")


/kaggle/working/model.h5

In [39]:
#create link for the model
FileLink("model_in_json.json")

/kaggle/working/model_in_json.json

In [40]:
#create link for the the vectorizer
FileLink("vector.pickel")

/kaggle/working/vector.pickel